## Stacking and Visualizing Data Example:

Here we stack galaxies by pre-determined properties and generate plots

In [ ]:
import yt
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import caesar
import pandas as pd
import csv
from scipy.stats import bootstrap
from functions import *

Here, we set the key parameters and point to our data files

In [ ]:
#Radial profiles (rp), Moment profiles (mp) and/or thermal energy (te)? 
analysis = 'te' 

label = 'test2' #Should match the label of the csv files generated in analysis_example.ipynb
output_label = 'test'

#Loading galaxy sample data
galaxies1 = pd.read_csv('galaxy_sample_%s.csv' %label, header=None)
sdat =np.array(galaxies1)
galaxies = pd.DataFrame(sdat)
stell = np.asarray(galaxies[0])
halo = np.asarray(galaxies[1])
rad = np.asarray(galaxies[2])
age = np.asarray(galaxies[3])
sfr = np.asarray(galaxies[4])
ssfr = [10**9*sfr[i]/stell[i] for i in range(len(stell))]

#CHANGE BELOW: Here you can set limits on your galaxy sample in terms of the properties listed above.
index_sample = [] #contains indices of your galaxy sample
for i in range(len(stell)):
    if stell[i]>10**(11) and age[i]>=1 and ssfr[i]<=0.01:
        index_sample.append(i)

If files are formatted and named correctly, you should not need to adjust anything in the section below, which walks through the three analyses.

In [ ]:
if 'rp' in analysis:
    loaded_data = pd.read_csv('radial_%s.csv'%label, header=None)
    if len(loaded_data) != len(galaxies):
        raise ValueError("Radial data and galaxy sample data are not the same size.")
    data_inbetween=np.array(loaded_data)
    radial_sample_prep = [data_inbetween[i] for i in index_sample]
    radial_sample = list(zip(*radial_sample_prep))

    #Stack and extract bootstrapped errors
    y_a = []
    err_a = []
    for i in radial_sample:
        y_a.append(np.mean(i))
        ii = (i,)
        bootstrap_ci = bootstrap(ii, np.mean, confidence_level=0.95,
                             random_state=1, method='percentile')
        err_a.append(bootstrap_ci.standard_error)
    #Background subtraction
    background_signal = np.mean(y_a[360:400])
    y_a = y_a-background_signal
    x_a = [(i*2.5)/60 for i in range(len(y_a))]

    plt.plot(x_a,y_a)
    plt.errorbar(x_a,y_a, yerr=err_a)
    plt.xlabel('Radius (arcmin)')
    plt.ylabel('Compton-y')
    plt.show()

    with open('stacked_radial_profiles_%s.csv' %output_label ,'a', newline = '') as f: 
        w = csv.writer(f)
        c = w.writerow(x_a)
        c = w.writerow(y_a)
        c = w.writerow(err_a)




In [ ]:
if 'mp' in analysis:
    a = pd.read_csv('m_0_%s.csv'%label, header=None)
    data =np.array(a)
    data2 = [data[i] for i in index_sample]
    b = pd.read_csv('m_1_%s.csv'%label, header=None)
    datb =np.array(b)
    datb2 = [datb[i] for i in index_sample]
    c = pd.read_csv('m_2_%s.csv'%label, header=None)
    datc =np.array(c)
    datc2 = [datc[i] for i in index_sample]

    data_a = list(zip(*data2))
    data_b = list(zip(*datb2))
    data_c = list(zip(*datc2))

    m_a = [np.mean(data_b[i])/np.mean(data_a[i]) for i in range(len(data_b))][0:300]
    m_b = [np.mean(data_c[i])/np.mean(data_a[i]) for i in range(len(data_c))][0:300]

    y_a = []
    for j in range(len(data[1])):
        y_a.append([])
        for i in range(4000):
            indices = np.random.choice(len(data_a[0]), len(data_a[0]), replace=True)
            new_b = [data_b[j][k] for k in indices]
            new_a = [data_a[j][k] for k in indices]
            y_a[j].append(np.mean(new_b)/np.mean(new_a))

    err_a = []
    for i in range(len(y_a)):
        err_a.append(np.std(y_a[i]))

    y_b = []

    for j in range(len(data[1])):
        y_b.append([])
        #print(j)
        for i in range(4000):
            indices = np.random.choice(len(data_a[0]), len(data_a[0]), replace=True)
            new_c = [data_c[j][k] for k in indices]
            new_a = [data_a[j][k] for k in indices]
            y_b[j].append(np.mean(new_c)/np.mean(new_a))
    err_b = []
    for i in range(len(y_b)):
        err_b.append(np.std(y_b[i]))

    xs = [i*2.5/60 for i in data[0]]
    xs = xs[0:300]

    plt.scatter(xs, m_a[0:300],s=30,color = 'red',label = 'm=1')
    plt.errorbar(xs, m_a[0:300],yerr = err_a,color = 'red',fmt ='none')
    plt.scatter(xs, m_b[0:300],s=30,color = 'orange',label = 'm=2')
    plt.errorbar(xs, m_b[0:300],yerr = err_b,color = 'orange',fmt ='none')
    plt.legend(fontsize = 40)
    plt.xlabel('Radius (arcmin)')
    plt.ylabel('$\Sigma(m)/\Sigma(m=0$)')
    plt.show()

    with open('stacked_moment_profiles_%s.csv' %output_label ,'a', newline = '') as f: 
        w = csv.writer(f)
        c = w.writerow(xs)
        c = w.writerow(m_a)
        c = w.writerow(err_a)
        c = w.writerow(m_b)
        c = w.writerow(err_b)



In [ ]:
if 'te' in analysis:
    z=0.9927
    comov = 3.3824
    theta = 2.5

    a = pd.read_csv('thermal_energy_%s.csv'%label, header=None)
    data_prep =np.array(a)

    data = [*data_prep[0],*data_prep[1],*data_prep[2]]
    therme = [2.9 * (comov/(1+z))**2 * (y2*(theta/60)**2)/(10**(-6)) for y2 in data] #Convert from summed aperature to thermal energy

    combined_data = [stell,halo,therme]
    thresholds_stellar = [10 ** x for x in [10.7, 10.9, 11.1, 11.3, 11.5, 11.7, 14.0]] #Change for bins of stellar mass
    thresholds_halo = [10 ** x for x in [12, 12.2, 12.4, 12.6, 12.8, 13, 13.2,13.4,13.6,13.8,16]] #Change for bins of stellar mass
    y,err,y2,err2 = make_thermal_energy_plot(combined_data, thresholds_stellar, thresholds_halo)

    plt.scatter(thresholds_stellar[1:-1] ,y, c = 'black')
    plt.errorbar(thresholds_stellar[1:-1],y, yerr=err, fmt='none', color = 'black')
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('Stellar Mass ($M_\odot$)')
    plt.ylabel('Thermal Energy ($10^{60}$ erg)')
    plt.show()

    plt.scatter(thresholds_halo[1:-1] ,y2, c = 'black')
    plt.errorbar(thresholds_halo[1:-1],y2, yerr=err2, fmt='none', color = 'black')
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('Halo Mass ($M_\odot$)')
    plt.ylabel('Thermal Energy ($10^{60}$ erg)')
    plt.show()

    with open('thermal_energy_plots_%s.csv' %output_label ,'a', newline = '') as f: 
        w = csv.writer(f)
        c = w.writerow(thresholds_stellar)
        c = w.writerow(y)
        c = w.writerow(err)
        c = w.writerow(thresholds_halo)
        c = w.writerow(y2)
        c = w.writerow(err2)
